# An introduction to SunPy for your heliophysics needs and use with Solar Orbiter,  Aditya-L1 and Proba3! 

<div>
<img src="./images/sunpy_logo.png" width="500" align="left"/>
</div>

# What is SunPy?

**SunPy** refers to both:

- the **SunPy Project**: an open, community-led effort to build and coordinate Python tools for solar and heliophysics research  
- the **`sunpy` package**: the core open-source Python library that provides foundational functionality for solar data analysis

In practice, when we say *“use SunPy”*, we often mean using the **SunPy ecosystem**: the `sunpy` package, and its affiliated packages plus closely related libraries (especially **Astropy**) that work together for searching, downloading, loading, plotting, and analysing data.

This ecosystem is especially useful when working across multiple missions and instruments, for example **Solar Orbiter** and **Aditya-L1**, because it helps standardise common workflows such as:

- querying data archives  
- reading common solar data formats  
- working with metadata and solar coordinate systems  
- visualising observations in a consistent way  

In this notebook, we’ll use the **`sunpy` package** (and key supporting libraries such as **Astropy**) to go through a short, practical SunPy workflow.

### We will cover
1. **Querying and downloading data** within the SunPy ecosystem using **Fido** (including the SOAR)
2. SunPy data containers: **`Map`** and **`TimeSeries`**
3. A quick introduction to SunPy’s **coordinates framework**


In [ ]:
from sunpy.net import Fido, attrs as a
import sunpy.map
from sunpy import timeseries as ts 
import sunpy_soar
import sunpy.data.sample 
from sunpy.time import parse_time

from astropy import units as u 
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import glob

 # 1. Finding and downloading data with SunPy (`Fido`)

## 1.1 What is `Fido`?

[`Fido`](https://docs.sunpy.org/en/stable/tutorial/acquiring_data/index.html) is SunPy’s unified interface for searching and downloading solar and heliophysics data.


In this hands-on tutorial, we’ll focus on **Solar Orbiter data access via SOAR**, but the same workflow works across many other data sources (e.g. SDO/AIA via the VSO).


In [ ]:
Fido

## 1.2 Searching with attributes

`Fido` searches are built by combining **attributes** from `sunpy.net.attrs` (often imported as `a`).

The most common ones you’ll use are:

- `a.Time(...)`
- `a.Instrument(...)`
- `a.Wavelength(...)` (when relevant)

We’ll start with a simple SDO/AIA example, then briefly show a Solar Orbiter/EUI query for comparison.


In [ ]:
result_aia = Fido.search(
    a.Time("2025-01-19T03:00:00", "2025-01-19T03:01:00"),
    a.Instrument("AIA"),
    a.Wavelength(171 * u.angstrom),
)
result_aia


## 1.3 Solar Orbiter data in SunPy: `sunpy-soar`

Solar Orbiter data are accessed via the **Solar Orbiter Archive (SOAR)**.

SunPy supports this through the external package **`sunpy-soar`**, which adds a SOAR client that plugs directly into the `Fido` interface.

That means Solar Orbiter searches look and feel the same as searches for other missions in the SunPy ecosystem.


## 1.3.1. Searching using `a.soar.Product`

`a.soar.Product` is the SOAR specific search attribute used to filter results to a particular Solar Orbiter data product

In [ ]:
a.soar.Product

In [ ]:
# a.soar.Product

In [ ]:
result = Fido.search(a.Time("2025-01-19 03:00", "2025-01-19 03:10"),
                     a.soar.Product.eui_fsi174_image, 
                     a.Level(2),
)
result


In [ ]:
result_2 = Fido.search(
    a.Time("2025-01-19 03:00", "2025-01-19 03:10"),
    a.Instrument("EUI"), 
    a.Provider.soar, 
)
result_2


## 1.3 Downloading the data with `Fido.fetch`

Once you have a search result from `Fido.search`, you can download the files with **`Fido.fetch`**.

`Fido.fetch` will download all files in the result (or a subset you choose). Under the hood it uses **parallel downloads** and will report any failed downloads so they can be retried.

In the example below, we’ll download just a small subset to keep things quick.


In [ ]:
eui_files = Fido.fetch(result)

In [ ]:
eui_files

## Here's how you can query and search for MAG data:

In [ ]:
mag_query = Fido.search(a.Time("2025-01-18", "2025-01-20"), 
                        a.soar.Product("MAG-RTN-NORMAL-1-MINUTE"), 
                        a.Level(2))

In [ ]:
mag_files = Fido.fetch(mag_query, path="./")

In [ ]:
mag_files

# 2. Data containers in SunPy

Now that we’ve seen how we can **search for and download data**, let’s look at how we can **load it into SunPy** for analysis and plotting.

SunPy provides core data container classes that offer a **consistent interface**, even when the underlying files come from different missions and instruments. These containers handle much of the mission-specific “file reading” and metadata handling for you.

The two main container types we’ll use are:

1. **`TimeSeries`**  (time-dependent measurements)
2. **`Map`** (2D solar images + WCS coordinates)

In the examples below, we’ll make one simple `TimeSeries`, then one `Map`.


## 2.1 `TimeSeries`

A [`TimeSeries`](https://docs.sunpy.org/en/stable/tutorial/timeseries.html) represents measurements as a function of time, and is backed by a `pandas.DataFrame`.

SunPy supports loading time series from a wide range of solar and heliophysics instruments. If you pass a supported file to `TimeSeries`, SunPy will automatically detect the data source and load the associated metadata.

Here we’ll load a simple example time series: **GOES X-ray flux**.


In [ ]:
goes_file = sunpy.data.sample.GOES_XRS_TIMESERIES
goes_file

In [ ]:
xrs = sunpy.timeseries.TimeSeries(goes_file)

In [ ]:
xrs.plot()

2.1.2 Inspect and manipulating the timeseries data

We can inspect and manipulate the timeseries, such as truncating (slicing) the data over a certain time period.

In [ ]:
xrs.meta

In [ ]:
xrs.units

In [ ]:
xrs.truncate("2011-06-07 05:00", "2011-06-07 08:00").plot()

## Solar Orbiter timeseries example

You can also pass a list of files to timeseries, and uses the `concatenate` keyword to create one continous timeseries. 

In [ ]:
mag_files

In [ ]:
mag_solo = sunpy.timeseries.TimeSeries(mag_files, concatenate=True)

In [ ]:
mag_solo.columns

In [ ]:
mag_solo.plot(columns=['B_RTN_0', 'B_RTN_1', 'B_RTN_2'])
plt.ylim(-20, 20)

## 2.2 `Map`

A [`Map`](https://docs.sunpy.org/en/stable/tutorial/maps.html) represents a 2D solar image together with its metadata and coordinate information (WCS).

One of the most useful features of `Map` is that it can read many different solar image files, and in particular **FITS files that follow common solar metadata conventions**. When you pass an image file to `sunpy.map.Map`, SunPy will automatically:

- read the image data  
- parse the FITS header and metadata  
- construct a WCS-aware `Map` object  

This makes it easy to plot images in a physically meaningful coordinate system and carry the observation information along with the data.

Next, we’ll load one example image into a `Map` and make a quick plot.


### Loading a `Map`

Once an image file has been downloaded, we can load it into a SunPy `Map`.  
A `Map` stores both:

- the **image data** (pixel values), and  
- the **metadata** (including pointing, plate scale, and coordinate information)

This means the image is no longer just “an array”, it becomes a solar-physics-aware object that knows where the Sun is in the field of view, what the pixel scale is, and how to interpret coordinates.


In [ ]:
eui_map = sunpy.map.Map(eui_files[0])

In [ ]:
eui_map.plot()

### Inspecting a `Map` and its attributes

A SunPy `Map` comes with a lot of useful information attached, for example:

- observation time (`m.date`)
- instrument and observatory (`m.instrument`)
- wavelength (`m.wavelength`)
- data (`m.data`)
- coordinate frame (`m.coordinate_frame`)
etc etc
These attributes make it much easier to do quick checks and avoid mistakes when working with multi-instrument datasets.


In [ ]:
eui_map.date

In [ ]:
eui_map.coordinate_frame

In [ ]:
eui_map.observer_coordinate

In [ ]:
eui_map.meta

### A note on WCS and WCSAxes (why SunPy plotting is special)

Most solar images are stored as pixel arrays, but for science we usually care about **physical coordinates** on the Sun (for example arcseconds, helioprojective coordinates, disk centre, limb position, etc).

SunPy `Map` objects include **WCS** information (World Coordinate System), which describes how pixel locations in the image correspond to real coordinates in the sky/solar frame.

When you plot a `Map`, SunPy uses Matplotlib’s **WCSAxes** system so that:

- the axes are labelled in **real coordinates**, not pixels
- coordinates remain meaningful even when you zoom, rotate, or overlay data

In short: WCS + WCSAxes is what allows SunPy plots to be physically interpretable, rather than just pretty images.


In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection=eui_map)
eui_map.plot(clip_interval=[1, 99.95]*u.percent)
eui_map.draw_limb()
eui_map.draw_grid(color='w')

coord = SkyCoord(500*u.arcsec, 1000*u.arcsec, frame=eui_map.coordinate_frame)
ax.plot_coord(coord, marker='x', color='r')

### Rotate a map

The `.rotate` method applies a rotation in the image plane, i.e. about an axis out of the page. In the case where we do not specify an angle (or rotation matrix), the image will be rotated such that the world and pixel axes are aligned. In the case of an image in helioprojective coordinate system, this means that solar north will be aligned with the y-like pixel axis of the image

In [ ]:
rotated_eui = eui_map.rotate(missing=eui_map.min())
rotated_eui.plot()
rotated_eui.draw_grid()

In [ ]:
bl = SkyCoord(-2000*u.arcsec, -2000*u.arcsec, 
              frame=rotated_eui.coordinate_frame)
tr = SkyCoord(2000*u.arcsec, 2000*u.arcsec, 
              frame=rotated_eui.coordinate_frame)
rotated_eui.submap(bl, top_right=tr).plot(clip_interval=[1, 99.95]*u.percent)

This is just the starting point, Map supports lots of common next steps like zooming into a flare region, transforming coordinates, aligning images, and overlaying context from other instruments. See the sunpy docs for more! 

# 3. Quick coordinates fun

## Plotting positions of spacecraft

Lets plot the positions of different spacecraft over the recent Solar Orbiter perihelion!

In [ ]:
import numpy as np
from sunpy.coordinates import get_horizons_coord, get_body_heliographic_stonyhurst
obstime = parse_time("2025-01-19")

In [ ]:
solo_coord = get_horizons_coord("solar orbiter", obstime)
earth_coord = get_body_heliographic_stonyhurst("earth", obstime)

In [ ]:
fig = plt.figure(dpi=120)
ax = fig.add_subplot(projection='polar')

# Transform to HGS
solo_coord_hgs = solo_coord
earth_coord_hgs = earth_coord


ax.plot(solo_coord_hgs.lon.to('rad'), solo_coord_hgs.radius,
        '.', markersize=10, label='SolO')
ax.plot(earth_coord_hgs.lon.to('rad'), earth_coord_hgs.radius,
        '.', markersize=10, label='Earth', color='green')
ax.plot(0, 0, marker='o', color='orange')
ax.legend(loc='lower right')
ax.set_theta_zero_location("S")
ax.set_title('Positions in Heliographic Stonyhurst (HGS) {:s}'.format(obstime.strftime("%Y-%m-%d")))

In [ ]:
aia_file = Fido.fetch(result_aia[0, 0], site="NSO")

In [ ]:
aia_map = sunpy.map.Map(aia_file)

In [ ]:
aia_map.plot()
eui_map.draw_limb(color='b', lw=2)

In [ ]:
aia_map.meta['rsun_ref'] = eui_map.meta['rsun_ref']

In [ ]:
eui_map_earth_view = eui_map.reproject_to(aia_map.wcs)
eui_map_earth_view.plot()
eui_map_earth_view.draw_limb(color='r')